# Retrieval and Generation with Bedrock Foundational Models

### Overview  
This notebook demonstrates how to perform retrieval-augmented generation (RAG) using Amazon Bedrock's foundational models. It covers retrieving relevant documents from a knowledge base and generating responses based on the retrieved context.

### Build your own Retrieval Augmented Generation (RAG) system
When constructing your own retrieval augmented generation (RAG) system, you can leverage a retriever system and a generator system. The retriever can be an embedding model that identifies the relevant chunks from the vector database based on similarity scores. The generator can be a Large Language Model (LLM) that utilizes the model's capability to answer questions based on the retrieved results (also known as chunks). In the following sections, we will provide additional tips on how to optimize the prompts for your RAG system.

In [ ]:
import advanced_rag_utils
import json
import importlib

# Reload module
importlib.reload(advanced_rag_utils)

# Re-import all functions
from advanced_rag_utils import *

from datetime import datetime, timedelta, UTC

notebook_start_time = datetime.now(UTC)
# Load variables from JSON file
with open("../variables.json", "r") as f:
    variables = json.load(f)

variables

## RAG with a simple question

##### We will ask the question "In text-to-sql, what are the stages in data generation process?" <br/>
##### We should expect a response from a PDF shown below that includes the three stages shown in picture below.
![Image](./image01.png)

### Configuration

In [ ]:
# Knowledge Base ID - Choose from different chunking strategies (Fixed, Hierarchical, or Semantic)
kb_id = variables["kbFixedChunk"] 

# Get the Bedrock Model ARN
model_id = get_model_arn(
    base_model_id="us.amazon.nova-lite-v1:0",
    #base_model_id="us.amazon.nova-pro-v1:0",
    account_number=variables['accountNumber'],
    region_name=variables['regionName']
)

# Number of relevant documents to retrieve for RAG
number_of_results = 5

# Create default generation configuration
generation_config = get_default_generation_config(
    max_tokens=4096,
    temperature=0,
    top_p=0.5
)

### Retrieve and Generate with a simple query

In [ ]:
# Define the query
# recollect in notebook 1.3.1 we executed the same query ? 
query  = "Who is the CEO, CFO, and CTO of Amazon? While answering the question, only use the data in context. If for any part of the question, you dont find the information in the context, please say I dont know for that part of the question."
# Perform retrieval-augmented generation (RAG)
response = retrieve_and_generate(
    query=query,
    kb_id=kb_id,
    model_id=model_id,
    number_of_results=number_of_results,
    generation_config=generation_config,
    region_name=variables['regionName']
)

# Display the results with citations
display_rag_results(response, show_citations=True)

### Comparison between chunking strategies: Fixed vs Semantic

##### Now, Let's ask a more nuanced question that needs to extract information from a table in the PDF. Also, let's ask it to do some analysis. <br/>
##### We will also compare the response quality when you use fixed size chunking vs Semantic chunking.
![image02](image02.png)

#### A nuanced query with a Fixed-sized chunking strategy

##### We will ask a question that should answer how net income changed from 2022 to 2023 to 2024.
![image03](image03.png)

In [ ]:
# Configuration for fixed chunking strategy
kb_id_fixed = variables["kbFixedChunk"]

# Model ID remains the same
model_id = get_model_arn(
    base_model_id="us.amazon.nova-lite-v1:0",
    account_number=variables['accountNumber'],
    region_name=variables['regionName']
)

In [ ]:
# Define the query for comparing net income changes
query = "In CONSOLIDATED STATEMENTS OF CASH FLOWS, How much did net income change in years 2022, 2023, 2024?"
# Perform RAG with fixed chunking strategy
response_fixed = retrieve_and_generate(
    query=query,
    kb_id=kb_id_fixed,
    model_id=model_id,
    number_of_results=number_of_results,
    generation_config=generation_config,
    region_name=variables['regionName']
)

# Display the results
display_rag_results(response_fixed)

#### The response above might not be accurate with what it should be.The accurate response should be:

> Year 2022 to Year 2023: \\$33,147 increase<br/>
Year 2023 to Year 2024: \\$28,823 increase 

#### Now Let's execute the same question while using the KB with Semantic Chunking.

In [ ]:
# Configuration for semantic chunking strategy
kb_id_semantic = variables["kbSemanticChunk"]

In [ ]:
# Enhance the query to request explanation of the calculation
query_with_explanation = "In CONSOLIDATED STATEMENTS OF CASH FLOWS, How much did net income change in years 2022, 2023, 2024? Show me how you did the math."
# Perform RAG with semantic chunking strategy
response_semantic = retrieve_and_generate(
    query=query_with_explanation,
    kb_id=kb_id_semantic,
    model_id=model_id,
    number_of_results=number_of_results,
    generation_config=generation_config,
    region_name=variables['regionName']
)

# Display the results
display_rag_results(response_semantic)

Compare the above results with the accurate response that should be:
> Year 2022 to Year 2023: \\$33,147 increase <br/>
> Year 2023 to Year 2024: \\$28,823 increase

As you can see here, Semantic Chunking was able to deliver accurate response as compared to Fixed Size chunking.

## Improve RAG quality with Enhanced Prompts

### Importance of Prompt Engineering
Prompt engineering refers to the practice of optimizing textual input to a large language model (LLM) to improve output and receive the responses you want. Prompting helps an LLM perform a wide variety of tasks, including classification, question answering, code generation, creative writing, and more. The quality of prompts that you provide to a LLM can impact the quality of the model's responses. <br/>
 

### Useful techniques to improve prompts for Amazon Nova models
Please refer [link](https://docs.aws.amazon.com/nova/latest/userguide/prompting.html) for the best practice of prompt engineering with Amazon Nova models. Following are a few highlights:
* Create precise prompts. Provide contextual information, speficy the output format and style, and provide clear prompt sections.
* Use system propmts to define how the model will repond.
* Give Amazon Nova time to think. For example, add ```"Think step-by-step."``` at the end of your query.
* Provide examples.

### Tips for using prompts in RAG
* Provide Prompt Template: As with other functionalities, enhancing the system prompt can be beneficial. You can define the RAG Systems description in the system prompt, outlining the desired persona and behavior for the model.
* Use Model Instructions: Additionally, you can include a dedicated ```"Model Instructions:"``` section within the system prompt, where you can provide specific guidelines for the model to follow. For instance, you can list instructions such as: ```In this example session, the model has access to search results and a user's question, its job is to answer the user's question using only information from the search results.```
* Avoid Hallucination by restricting the instructions: Bring more focus to instructions by clearly mentioning "DO NOT USE INFORMATION THAT IS NOT IN SEARCH RESULTS!" as a model instruction so the answers are grounded in the provided context.


#### Without a Prompt Template

In [ ]:
# Define the query about Amazon's financial results
query = "Show me the amazon financial results for 2023"

# Perform RAG without prompt template
response_no_template = retrieve_and_generate(
    query=query,
    kb_id=kb_id,
    model_id=model_id,
    number_of_results=number_of_results,
    generation_config=generation_config,
    region_name=variables['regionName']
)

# Display the results
display_rag_results(response_no_template)

#### Using a Prompt Template

In [ ]:
# Define a prompt template for financial analysis
prompt_template = """
You are a professional financial analyst. 
Based on the retrieved content from Amazon's 10-K filings, provide clear, concise, and insightful answers to user questions. 
When summarizing financial results, respond in bullet points highlighting key metrics, trends, and takeaways. 
Ensure your answers are accurate, data-driven, and easy to understand.
Format the output as Markdown document.

$Query$
Resource: $search_results$
"""

# Perform RAG with the prompt template
response_with_template = retrieve_and_generate(
    query=query,
    kb_id=kb_id,
    model_id=model_id,
    number_of_results=number_of_results,
    generation_config=generation_config,
    prompt_template=prompt_template,
    region_name=variables['regionName']
)

# Display the results as Markdown
# display_rag_results(response_with_template, format_as_markdown=True)
print('----------------- Answer ---------------------')
from IPython.display import display, Markdown
display(Markdown(response_with_template['output']['text'].replace("$", "USD ")))

#### Change the prompt to produce JSON output

In [ ]:
# Modify the prompt template to request JSON output
json_prompt_template = """
You are a professional financial analyst. 
Based on the retrieved content from Amazon's 10-K filings, provide clear, concise, and insightful answers to user questions. 
When summarizing financial results, respond in bullet points highlighting key metrics, trends, and takeaways. 
Ensure your answers are accurate, data-driven, and easy to understand.
Format the output as JSON document.

$Query$
Resource: $search_results$
"""

# Perform RAG with JSON prompt template
response = retrieve_and_generate(
    query=query,
    kb_id=kb_id,
    model_id=model_id,
    number_of_results=number_of_results,
    generation_config=generation_config,
    prompt_template=json_prompt_template,
    region_name=variables['regionName']
)

# Display the results as Markdown to properly format the JSON
print('----------------- Answer ---------------------')
from IPython.display import display, Markdown
display(Markdown(response['output']['text'].replace("$", "\\$")))
#display_rag_results(response_json, format_as_markdown=True)

### Cost Summary for Running This Notebook
In this notebook, we used two LLMs: 1) Embedding 2) Text Generation.

Let us breakdown the cost

In [ ]:
embedding_model_id = "amazon.titan-embed-text-v2:0"
inference_model_id = "us.amazon.nova-lite-v1:0"

# Mark end of query executions here:
notebook_end_time = datetime.now(UTC)

In [ ]:
print(notebook_start_time, notebook_end_time)

embedding_cost = get_bedrock_token_based_cost(embedding_model_id, notebook_start_time, notebook_end_time)
inference_cost = get_bedrock_token_based_cost(inference_model_id, notebook_start_time, notebook_end_time)

In [ ]:
embedding_cost

In [ ]:
inference_cost